In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
sns.set(style='ticks')
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv")
df.columns = df.columns.str.replace(' ', '') 
print("\n\n", df) 

In [ ]:
df.info()


# **The first question we can try and explore in the given dataset is what is the average Annual Income based on gender??**

In [ ]:
x = df.groupby('Gender')['AnnualIncome(k$)'].mean()
print(x)


In [ ]:
fig = plt.figure()
gender = ['Female','Male']
ax = fig.add_axes([0,0,0.4,0.4])
ax.bar(gender,x)

# **Clearly the males have a greater annual income on an average, as can be seen in the graph above.**

# The next thing we can explore is the relationship between the annual income and the spending score based on gender 

In [ ]:
fg = sns.FacetGrid(data=df, hue='Gender', hue_order=gender, aspect=2.0)
fg.map(plt.scatter, 'AnnualIncome(k$)', 'SpendingScore(1-100)').add_legend()

# We get an idea based on the scatter plot above that maybe we can make clusters of the different customers..

In [ ]:
# let's standardize the variables before using any algorithm on them.
scaler = StandardScaler()
features = df.iloc[:200,2:5]
scaled_features = scaler.fit_transform(features)

In [ ]:
kmeans_kwargs = {
                "init": "random",
                "n_init": 10,
                "max_iter": 300,
                "random_state": 42,
                 }

sse = []

for k in range(1,11):
    kmeans = KMeans(n_clusters = k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

# Based on the plot above k = 4 can be chosen as the optimal number of clusters.

In [ ]:
kmeans = KMeans(init = "random" , n_clusters = 4 , n_init = 10, max_iter = 300 , random_state = 42)
kmeans.fit(scaled_features)
y_kmeans = kmeans.predict(scaled_features)

In [ ]:
# the lowest SSE value..
kmeans.inertia_


In [ ]:
 ##final location of the centroids of the clusters.
kmeans.cluster_centers_

In [ ]:
# number of iterations for convergence..
kmeans.n_iter_

In [ ]:
##The cluster assignments are saved in a one-dimenstional numpy array
kmeans.labels_[:5]

In [ ]:
plt.scatter(scaled_features[:, 0], scaled_features[:, 1], c=y_kmeans, s=50, cmap='viridis')
plt.xlabel("Annual Income")
plt.ylabel("Spending Score")
plt.legend()

# Though there is some overlap but we can clearly see the four separate clusters.Now let us explore the separation between the clusters further based on Annual Income and Spending Score of Customers via plotting Boxplots.

In [ ]:
ax = sns.boxplot(x = kmeans.labels_ , y = "AnnualIncome(k$)", data = df)

In [ ]:
ax = sns.boxplot(x = kmeans.labels_ , y = "SpendingScore(1-100)", data = df)

# Clearly, we can see that the clusters are separated very well by the Spending Score. There is a slight overlap between clusters in the case of separation based on the annual Income.